<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Project Fire Risk

_by Jeremy Freeman (CarbonPlan), November 28, 2020_

This notebook shows a couple different ways of extracting the fire risk for a forest carbon project
based on historical records.


In [ ]:
import fsspec
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gp
import regionmask as rm
from scipy.stats import binom
from shapely.geometry import Point

In [ ]:
%matplotlib inline
from showit import image
import matplotlib.pyplot as plt

First we load the monthly MTBS data


In [ ]:
mapper = fsspec.get_mapper(
    "/Users/freeman/workdir/carbonplan-data/processed/mtbs/conus/4000m/monthly.zarr"
)
da = xr.open_zarr(mapper)["monthly"]
da.load()

We'll also load "supersections" which we'll use for computing regional statistics


In [ ]:
regions = gp.read_file(
    "/Users/freeman/workdir/carbonplan-data/raw/ecoregions/supersections.geojson"
)

And we'll compute masks from these regions


In [ ]:
masks = rm.mask_3D_geopandas(regions, da)

Now we'll demo two examples of aggregating within regions to extract historical fire statistics.


In [ ]:
def integrated_risk(p):
    return (1 - binom.cdf(0, 100, p)) * 100

## Querying a project based on ecoregion


First we find the ecoregion a project belongs to


In [ ]:
p = Point(-121.183861, 42.672889)

In [ ]:
ecoregion = [i for i in regions.index if regions.geometry[i].contains(p)][0]

Get integrated fire risk


In [ ]:
integrated_risk(
    da.where(masks[ecoregion])
    .mean(["x", "y"])
    .groupby("time.year")
    .sum()
    .sel(year=slice("2001", "2018"))
    .mean()
)

Plot burn area over time


In [ ]:
da.where(masks[ecoregion]).mean(["x", "y"]).groupby("time.year").sum().sel(
    year=slice("1984", "2018")
).plot()

## Querying a project based on location


In this version we define a disc with fixed radius at the center of a location and integrate over
that


In [ ]:
center = [-121.183861, 42.672889]

In [ ]:
lat = da["lat"]
lon = da["lon"]
dist = (lon - center[0]) ** 2 + (lat - center[1]) ** 2
mask = dist < 0.5

In [ ]:
integrated_risk(
    da.where(mask)
    .mean(["x", "y"])
    .groupby("time.year")
    .sum()
    .sel(year=slice("2001", "2018"))
    .mean()
)

## Querying a project based on the shapefile


Finally, we can use a shapefile for a project and integrate over that area


In [ ]:
region = gp.read_file("CAR1314.json")

In [ ]:
mask = rm.mask_3D_geopandas(region1.simplify(0.002).buffer(0.005), da)[0]

In [ ]:
integrated_risk(
    da.where(mask)
    .mean(["x", "y"])
    .groupby("time.year")
    .sum()
    .sel(year=slice("2001", "2018"))
    .mean()
)

In [ ]:
da.where(mask).mean(["x", "y"]).groupby("time.year").sum().plot()